In [119]:
import numpy as np
import astropy.units as u
import os, fnmatch
import sunpy.coordinates
import pandas as pd
import shutil
import math
import random
import pickle
import warnings
import sunpy.map
import sys
import imageio
import matplotlib as plt
import matplotlib.pyplot as pyplot
from datetime import datetime, timedelta
from astropy.coordinates import SkyCoord
from sunpy.coordinates import frames
from PIL import Image, ImageOps, ImageDraw, ImageFont
from astropy.io import fits
from skimage import img_as_ubyte

# Generating Disks

### Author: Alex Feghhi 
### Created: 7/30/2019
### Last run: 1/24/2020

### Goals:
- Reads in a data text file containing relevant keywords
- Outputs time-labeled disks from the given start to end dates

Generate work directory and directory to store output files

In [66]:
def verify_dir(dir_name):
    if not os.path.exists(dir_name):
            os.mkdir(dir_name)
            print("Directory {} does not exist. Creating...".format(dir_name))

savedir = 'C:/Users/alexf/Desktop/HMI_Data'
verify_dir(savedir)
sharp_dir = os.path.join(savedir, 'sharp')
verify_dir(sharp_dir)
Ydata_dir =  os.path.join(savedir, 'Ydata')
verify_dir(Ydata_dir)
Xdata_dir = os.path.join(savedir, 'Xdata')
verify_dir(Xdata_dir)

Open and parse data into a Pandas dataframe

Data text file generated through downloading sharp files from jsoc and extracting keywords. Jupyter notebook here: (link to Jupyter notebook)

In [67]:
def load_df(df_path):#load data text file
    df = pickle.load(open(df_path,'rb'))
    times = df['T_REC'].sort_values(ascending=True)
    print('The dataframe you have loaded contains HARPS from {} to {}'.format(times.iloc[0],times.iloc[-1]))
    return df

df = load_df((os.path.join(savedir,'HARP_df.pickle')))#dataframe must be loaded before generating any disks
df

The dataframe you have loaded contains HARPS from 2010.05.01_00:00:00_TAI to 2010.12.01_00:00:00_TAI


,HARPNUM,T_REC,NAXIS1,NAXIS2,CDELT1,CDELT2,CRPIX1,CRPIX2,IMCRPIX1,IMCRPIX2,LAT_FWT,LON_FWT,NPIX,FILEDIR
0,1,2010.05.01_00:00:00_TAI,207,279,0.504348,0.504348,-1560.519043,945.743408,2037.480957,2049.743408,23.815,-78.1948,3842,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
1,1,2010.05.01_01:00:00_TAI,209,276,0.504346,0.504346,-1556.453247,945.827393,2037.546753,2049.827393,23.7317,-77.6274,4671,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
2,1,2010.05.01_02:00:00_TAI,214,276,0.504344,0.504344,-1550.451538,946.790283,2037.548462,2049.790283,23.6871,-77.0768,5143,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
3,1,2010.05.01_03:00:00_TAI,218,276,0.504344,0.504344,-1545.511353,947.797119,2037.488647,2049.797119,23.7674,-76.8241,5481,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
4,1,2010.05.01_04:00:00_TAI,223,276,0.504344,0.504344,-1539.525635,948.895264,2037.474365,2049.895264,23.8001,-76.1515,5707,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
5,1,2010.05.01_05:00:00_TAI,227,276,0.504344,0.504344,-1533.547119,949.906006,2037.452881,2049.906006,23.8457,-75.7027,5881,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
6,1,2010.05.01_06:00:00_TAI,233,275,0.504345,0.504345,-1526.512573,950.892822,2037.487427,2049.892822,23.8067,-74.9708,5978,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
7,1,2010.05.01_07:00:00_TAI,237,275,0.504346,0.504346,-1520.547729,951.898438,2037.452271,2049.898438,23.7827,-74.4586,6039,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
8,1,2010.05.01_08:00:00_TAI,241,275,0.504346,0.504346,-1514.553589,952.929932,2037.446411,2049.929932,23.8274,-73.74,6014,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...
9,1,2010.05.01_09:00:00_TAI,246,275,0.504346,0.504346,-1507.537109,953.945801,2037.462891,2049.945801,23.8,-73.084,5892,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...


Generate a radius dictionary by finding the max number of pixels per harp, then the maximum height/width at the time, and then dividing by 2.

NPIX is number of pixels, NAXIS1 and NAXIS2 are height and width in pixels

In [68]:
def generate_max_radius_rows(df):
    max_radius_rows = df[df.groupby(['HARPNUM'])['NPIX'].transform(max) == df['NPIX']]#fix iloc issue
    max_radius_rows = max_radius_rows.reset_index(drop = True)
    return max_radius_rows#dataframe containing each harp at its max size

def generate_radius_dict(max_radius_rows):
    max_radius_rows['RADIUS'] = max_radius_rows.loc[:,['NAXIS1', 'NAXIS2']].max(axis=1)/2#find the max height/width then divide by 2
    radius_dict = pd.Series(max_radius_rows.RADIUS.values,index=max_radius_rows.HARPNUM).to_dict()#dict of each max radius
    return radius_dict

max_radius_rows = generate_max_radius_rows(df)#run if generating ellipses at max size
radius_dict = generate_radius_dict(max_radius_rows)#run if generating circles at max size
max_radius_rows

,HARPNUM,T_REC,NAXIS1,NAXIS2,CDELT1,CDELT2,CRPIX1,CRPIX2,IMCRPIX1,IMCRPIX2,LAT_FWT,LON_FWT,NPIX,FILEDIR,RADIUS
0,1,2010.05.06_11:00:00_TAI,534,237,0.504350,0.504350,60.150391,1008.270264,2037.150391,2050.270264,24.5978,-7.00541,71540,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,267.0
1,10,2010.05.03_17:00:00_TAI,292,133,0.504353,0.504353,-104.699951,-656.175293,2037.300049,2049.824707,-26.3459,-8.64431,27332,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,146.0
2,11,2010.05.04_09:00:00_TAI,216,151,0.504345,0.504345,1648.242798,707.059082,2037.242798,2050.059082,17.5913,58.1293,7488,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,108.0
3,12,2010.05.09_18:00:00_TAI,503,222,0.504352,0.504352,207.957642,-434.730225,2036.957642,2050.269775,-20.0694,-0.844332,65092,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,251.5
4,14,2010.05.06_03:00:00_TAI,160,84,0.504340,0.504340,425.229980,824.293945,2037.229980,2050.293945,20.9194,11.1182,8777,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,80.0
5,15,2010.05.05_18:00:00_TAI,62,39,0.504351,0.504351,976.199707,565.992432,2037.199707,2049.992432,13.7271,30.5712,1495,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,31.0
6,16,2010.05.06_01:00:00_TAI,173,68,0.504340,0.504340,824.267090,482.198975,2037.267090,2050.198975,10.4225,23.2908,6855,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,86.5
7,17,2010.05.07_16:00:00_TAI,160,92,0.504353,0.504353,854.069458,-540.789795,2037.069458,2050.210205,-21.0808,26.0136,8632,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,80.0
8,187,2010.10.01_00:00:00_TAI,856,448,0.504297,0.504297,1365.100830,767.709473,2038.100830,2049.709473,21.4595,37.0992,118332,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,428.0
9,19,2010.05.13_14:00:00_TAI,114,59,0.504354,0.504354,364.779785,974.542969,2036.779785,2050.542969,27.3063,10.2674,3535,C:/Users/alexf/Desktop/HMI_Data/sharp\hmi.shar...,57.0


Set time interval

In [111]:
time_interval = timedelta(minutes = 60)

Helper functions for generating savedirs

In [70]:
def generate_sizedirs(sizes,dir_name):
    for size in sizes:
        data_dir = dir_name + '_' + str(size)
        verify_dir(data_dir)

Helper functions for bitmap generation

In [95]:
def get_bitmap(fits_file_path):#run ignore warnings to kill warnings from the second line of this function 
    bitmap = sunpy.map.Map(fits_file_path)
    bitmap = bitmap.data
    bitmap[np.add(bitmap == 1,bitmap == 2)] = 0#set off_harp values
    off_harp = (bitmap == 0)
    on_harp = np.logical_not(off_harp)
    bitmap[on_harp] = 1#everything else set to 1
    return bitmap

def edge_check(xc,yc,width,height):
    if (xc < 0):
        xc = 0
    if (yc < 0):
        yc = 0
    if (xc + width > 4095):
        xc = 4095 - width
    if (yc + height > 4095):
        yc = 4095 - height
    return xc,yc

def plot_bitmap(xc,yc,bitmap,layer):
    yc = 4095 - yc
    xc = int(xc)
    yc = int(yc)
    layer[yc-bitmap.shape[0]:yc,xc:xc+bitmap.shape[1]] = bitmap
    return layer

def set_to_binary(array):
    array_avg = np.mean(array)
    array[array > array_avg] = 1
    array[array <= array_avg] = 0
    return array

def generate_array_list(image_list,size):
    return [np.array(Image.fromarray(layer).resize((size,size),Image.LANCZOS)) for layer in image_list]

def generate_cropped_array_list(image_list,size):
    return [np.array(Image.fromarray(layer).crop((CROP_INDEX,CROP_INDEX,4096-CROP_INDEX,4096-CROP_INDEX)).resize((size,size),Image.LANCZOS)) for layer in image_list]

def save_bitmap_stack(image_list,current_time,size,dir_name,cropped):#refactor
    data_dir = dir_name + '_' + str(size)
    if not cropped:
        array_list = generate_array_list(image_list,size)
        savedir = os.path.join(data_dir,current_time.strftime('%Y%m%d_%H%M%S') + '_' + str(size))
        np.save(savedir, np.stack([set_to_binary(array) for array in array_list],axis=0))
    else:
        array_list = generate_cropped_array_list(image_list,size)
        savedir = os.path.join(data_dir,current_time.strftime('%Y%m%d_%H%M%S') + '_' + str(size))
        np.save(savedir, np.stack([set_to_binary(array) for array in array_list],axis=0))

Bitmap generation function

In [117]:
def generate_filled_bitmap(start,end,sizes,dir_name,cropped = False):
    generate_sizedirs(sizes,dir_name)
    number_disks = int((end-start)/time_interval + 1)
    print('Generating filled_bitmaps: cropped = {}, {} thru {} at sizes {}'.format(cropped,start,end,sizes))
    warnings.simplefilter("ignore")
    for i in range(number_disks):
        current_time = start + i*time_interval
        timestring = current_time.strftime('%Y.%m.%d_%X_TAI')
        rows = df.loc[df['T_REC'] == timestring].reset_index()
        if (not(rows.empty)):
            image_list = []
            for index, row in rows.iterrows():#iterate over rows
                bitmap_dir = row['FILEDIR']
                bitmap = get_bitmap(bitmap_dir)
                xc = row['CRPIX1'] + row['IMCRPIX1']
                yc = row['CRPIX2'] + row['IMCRPIX2']
                width = bitmap.shape[0]
                height = bitmap.shape[1]
                xc,yc = edge_check(xc,yc,width,height)
                layer = np.zeros((4096,4096))
                layer = plot_bitmap(xc,yc,bitmap,layer)
                layer = np.rot90(layer,2)
                image_list.append(layer)
            for size in sizes:
                save_bitmap_stack(image_list,current_time,size,dir_name,cropped)

Generate the bitmaps

In [118]:
start = datetime(2010, 10, 25,20,0,0)#date time object format is year, month, day, hour, minute, second
end = datetime(2010,11, 1, 0,0,0)#the end time is included amongst disks generated
sizes = [256]

generate_filled_bitmap(start,end,sizes,os.path.join(Ydata_dir,'filled_bitmap_uncropped'))

Generating filled_bitmaps: cropped = False , 2010-10-25 20:00:00 thru 2010-11-01 00:00:00 at sizes [256]


Testing

Change bitmap helper functions

Load and test bitmaps

In [102]:
df_index = 0
bitmap_dir = df.loc[df_index,['FILEDIR']][0]
bitmap = get_bitmap(bitmap_dir)
zeros = np.where(bitmap == 0)[0].shape[0]
ones = np.where(bitmap == 1)[0].shape[0]
width = df.loc[df_index,['NAXIS1']][0]
height = df.loc[df_index,['NAXIS2']][0]
print(zeros)
print(ones)
print(zeros + ones)
print(width * height)

53911
3842
57753
57753


Video generation

In [1]:
source_X_dir = os.path.join(Xdata_dir,'uncropped_hmis_256')
source_Y_dir = os.path.join(Ydata_dir,'bitmap_uncropped_256')

NameError: name 'os' is not defined

In [7]:
def _image_preprocessing(filename, im_type, xsize, ysize):   #ignoring xsize and ysize since using 256x256 images
    im = np.load(filename)
    if (im_type == "Y"):    #aggregate and orient all layers of the synthetic image
        im = np.max(im, axis=0)
        # im = np.flip(np.max(im, axis=0), axis = 0) - use this if the image needs to be flipped
    mn = np.amin(im)
    mx = np.amax(im)
    norm_im = (im-mn)/(mx-mn)
    return np.dstack([norm_im, norm_im, norm_im])

def _npy_of_text(text,width,height):
    font_dir = '/home6/gmackint/nobackup/DL/HMI-AI/tensorflow-pix2pix-master/'
    img = Image.new('RGB', (height, width), color=0)   #black background
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_dir+'arial.ttf', 10)
    color = 'rgb(255, 255, 255)' # white text
    draw.text((0,0), text, fill=color, font=font)
    im = np.array(img)
    mn = np.amin(im)
    mx = np.amax(im)
    norm_im = (im-mn)/(mx-mn)
    return norm_im

def _add_label_to_frame(frame, text):
    txt_box_height = 15
    txt_box_width = frame.shape[1]
    frame[0:txt_box_height,0:txt_box_width] = _npy_of_text(text,txt_box_height,txt_box_width)
    return frame

TypeError: 'module' object is not callable

In [ ]:
# Create and save dictonary of HMI files and mapped "Y" images to be used for this model

HMI_syn_images = []
chron_order = 0
forecast = 0    # number of days to predict into the future
Xnames = sorted(os.listdir(source_X_dir))

for i in range(0, len(Xnames)):
    #fix
    future_date = datetime.strptime(Xnames[i][0:8], '%Y%m%d') + timedelta(days=forecast)   
    Yname = future_date.strftime('%Y%m%d') + Xnames[i][8:]
    if os.path.isfile(source_Y_dir+Yname):
        HMI_syn_images.append([chron_order, 0, Xnames[i], Yname ])   # second field will be set as sequence number for training once the list is randomized
        chron_order = chron_order +1
    
print ("Ready to process ", len(Xnames), " files, of which ", chron_order, " have matching Y images.")

In [ ]:
# Create the X/Y numpy arrays for model training and testing

sample_size = len(HMI_syn_images)

# percent of total dataset for training
percent_training = 10
training_end = round(sample_size*(percent_training/100))

HMI_training_images = HMI_syn_images[:training_end]
training_dataset_X = np.zeros((training_end, 256, 256, 3))
training_dataset_Y = np.zeros((training_end, 256, 256, 3))

HMI_testing_images = HMI_syn_images[training_end+1:sample_size]
testing_dataset_X = np.zeros((sample_size-training_end, 256, 256, 3))
testing_dataset_Y = np.zeros((sample_size-training_end, 256, 256, 3))

#Randomize the training dataset list, but leave the test dataset in chronological order
random.shuffle(HMI_training_images)

for i in range(0, len(HMI_training_images)):
    HMI_training_images[i][1] = i   #record the training sequence number for later sorting
    training_dataset_X[i] = _image_preprocessing(source_X_dir+HMI_training_images[i][2], "X", 256, 256)
    training_dataset_Y[i] = _image_preprocessing(source_Y_dir+HMI_training_images[i][3], "Y", 256, 256)

for i in range(0, len(HMI_testing_images)):
    HMI_testing_images[i][1] = i   #record the sequence number for later sorting - somewhat redundant
    testing_dataset_X[i] = _image_preprocessing(source_X_dir+HMI_testing_images[i][2], "X", 256, 256)
    testing_dataset_Y[i] = _image_preprocessing(source_Y_dir+HMI_testing_images[i][3], "Y", 256, 256)
    


In [ ]:
# Quick visual check that the images are correct

check_image = 200

img_size = 8

if (check_image < len(testing_dataset_X)):
    print ("Image %d \n   Showing X %s \n  and Y %s\n" %(check_image, HMI_syn_images[check_image][2], HMI_syn_images[check_image][3]))
    fig = plt.figure()
    fig.tight_layout()
    fig.set_size_inches(2*img_size, img_size)
    fig.add_subplot(1, 2, 1)
    plt.imshow(testing_dataset_X[check_image]) 
    plt.axis('off')
    fig.add_subplot(1, 2, 2)
    plt.imshow(testing_dataset_Y[check_image])
    plt.axis('off')
    plt.show()
else:
    print ("Image %d is out of range for the test data set  (max=%d)\n" %(check_image, len(testing_dataset_X)))

    

In [13]:
# optionally create a video to check X-Y training and test data
frame_rate = 30
txt_height = 10

# sort metadata list by chronological sequence (field 0) to produce time series for video
HMI_training_images.sort(key=lambda x: x[0])
HMI_testing_images.sort(key=lambda x: x[0])

X_vid = np.zeros(training_dataset_X.shape)
Y_vid = np.zeros(training_dataset_Y.shape)

# Print file headers on the X and Y training images
for i in range(len(HMI_training_images)):
    X_vid[i] = _add_label_to_frame(training_dataset_X[HMI_training_images[i][1]], HMI_training_images[i][2][11:11+15])

for i in range(len(HMI_training_images)):
    Y_vid[i] = _add_label_to_frame(training_dataset_Y[HMI_training_images[i][1]], HMI_training_images[i][3][:15])

# create composite video of training set
frame = 256
composite_video = np.zeros((len(X_vid), frame, frame*2, 3))  # two images across
composite_frame = np.zeros((frame, frame*2, 3))

image_1_offset = 0
image_2_offset = image_1_offset+frame

for i in range(len(X_vid)):
    composite_frame[0:frame, image_1_offset:image_1_offset+frame] = X_vid[i]
    composite_frame[0:frame, image_2_offset:image_2_offset+frame] = Y_vid[i]
    composite_video[i] = composite_frame
    
imageio.mimwrite(numpy_dir+'images/XY_training_verification.mp4', img_as_ubyte(composite_video) , fps = frame_rate)


# create composite video of test set
X_vid = np.zeros(testing_dataset_X.shape)
Y_vid = np.zeros(testing_dataset_Y.shape)

# Print file headers on the X and Y testing images
for i in range(len(HMI_testing_images)):
    X_vid[i] = _add_label_to_frame(testing_dataset_X[HMI_testing_images[i][1]], HMI_testing_images[i][2][11:11+15])

for i in range(len(HMI_testing_images)):
    Y_vid[i] = _add_label_to_frame(testing_dataset_Y[HMI_testing_images[i][1]], HMI_testing_images[i][3][:15])

# create composite video of test set
frame = 256
composite_video = np.zeros((len(X_vid), frame, frame*2, 3))  # two images across
composite_frame = np.zeros((frame, frame*2, 3))

image_1_offset = 0
image_2_offset = image_1_offset+frame

for i in range(len(X_vid)):
    composite_frame[0:frame, image_1_offset:image_1_offset+frame] = X_vid[i]
    composite_frame[0:frame, image_2_offset:image_2_offset+frame] = Y_vid[i]
    composite_video[i] = composite_frame
    
imageio.mimwrite(numpy_dir+'images/XY_testing_verification.mp4', img_as_ubyte(composite_video) , fps = frame_rate)